In [1]:
pip install cirq -q

Note: you may need to restart the kernel to use updated packages.


# Clock, Shift, and Fourier Operators

In quantum computing, there are generalizations of qubits (two-level systems) such as spin-$1/2$ particles to *qudits* (d-level systems) such as higher spin particles. To generalize quantum computing to this scenario, we need to generalize the gates or operators acting on qubits (or density matrices). Implementing these gates is a necessary step in developing a generalization of error correcting codes known as *surface codes*. In particular, if we wish to model the quantum information of systems with various particles of differing spin, that is $d$-level systems represented by qudits of differing dimensions, we must generalize not only the gates operating on individual qudits, but also those multi-qudit gates that model entanglement of two or more qudits. 

The generalization of single qudit gates involves constructing several gates that form a universal gate set for quantum computation. In this notebook, we construct several of these gates and create a sufficient list of gates for quantum computing. We begin with the following two gates. 

Shift (or generalized $X$-gate) operator matrix:

$$
S =
\begin{pmatrix}
0 & 1 & 0 & \cdots & 0 \\
0 & 0 & 1 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & 1 \\
1 & 0 & 0 & \cdots & 0
\end{pmatrix}
$$

Note, some authors may use the transpose of this matrix as the shift operator. The next gate we need is the, Fourier (or generalized Hadamard gate) operator matrix:

$$
C =
\begin{pmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & e^{2πi/n} & e^{4πi/n} & \cdots & e^{2(n-1)πi/n} \\
1 & e^{4πi/n} & e^{8πi/n} & \cdots & e^{4(n-1)πi/n} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & e^{2(n-1)πi/n} & e^{4(n-1)πi/n} & \cdots & e^{2(n-1)^2πi/n}
\end{pmatrix}
$$

Here is some code to construct these two matrix gates using Cirq. 

In [2]:
import cirq
import numpy as np

def get_shift_matrix(n: int) -> np.ndarray:
    S = np.zeros((n, n), dtype=np.complex128)
    for i in range(n):
        S[(i + 1) % n][i] = 1
    return S

def get_clock_matrix(n: int) -> np.ndarray:
    C = np.zeros((n, n), dtype=np.complex128)
    for i in range(n):
        for j in range(n):
            C[i][j] = np.exp(2j * np.pi * i * j / n)
    return C

# Test the code for n = 3
n = 5
S = get_shift_matrix(n)
C = get_clock_matrix(n)
print(f"Shift matrix:\n{S}\n")
print(f"Clock matrix:\n{C}\n")


Shift matrix:
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]]

Clock matrix:
[[ 1.        +0.j          1.        +0.j          1.        +0.j
   1.        +0.j          1.        +0.j        ]
 [ 1.        +0.j          0.30901699+0.95105652j -0.80901699+0.58778525j
  -0.80901699-0.58778525j  0.30901699-0.95105652j]
 [ 1.        +0.j         -0.80901699+0.58778525j  0.30901699-0.95105652j
   0.30901699+0.95105652j -0.80901699-0.58778525j]
 [ 1.        +0.j         -0.80901699-0.58778525j  0.30901699+0.95105652j
   0.30901699-0.95105652j -0.80901699+0.58778525j]
 [ 1.        +0.j          0.30901699-0.95105652j -0.80901699-0.58778525j
  -0.80901699+0.58778525j  0.30901699+0.95105652j]]



The *clock matrix* is denoted by $C$ and is a diagonal matrix with entries $e^{2\pi i n/N}$, where $n$ runs from $0$ to $N-1$ and $N$ is the size of the system. In other words, the diagonal entries of the clock matrix are the $N$th roots of unity. The general form of the clock matrix is:

$$
\begin{pmatrix}
1 & 0 & 0 & \cdots & 0 \\
0 & e^{2\pi i/N} & 0 & \cdots & 0 \\
0 & 0 & e^{4\pi i/N} & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & e^{2\pi i (N-1)/N} \\
\end{pmatrix} $$


Note that the shift matrix is a cyclic permutation matrix that corresponds to shifting the system by one site, while the clock matrix contains information about the periodicity of the system. Together, the clock and shift matrices generate the so-called "Heisenberg group," which is a fundamental mathematical structure that arises in the study of quantum mechanics on periodic lattices.

Here we implement Cirq code to take as input the dimension n, and give as output the clock and shift matrix as operators on a qudit of dimension n. 

In [3]:
import cirq
import numpy as np

def clock_shift_matrices(n: int):
    # Define the clock matrix
    clock_mat = np.zeros((n, n), dtype=np.complex128)
    for i in range(n):
        clock_mat[i, i] = np.exp(2j * np.pi * i / n)
    clock_op = cirq.MatrixGate(clock_mat, qid_shape=(n,))

    # Define the shift matrix
    shift_mat = np.roll(np.eye(n), -1, axis=0)
    shift_op = cirq.MatrixGate(shift_mat, qid_shape=(n,))

    return clock_op, shift_op


The only change we made is to add the qid_shape argument with value (n,) to the MatrixGate constructor.

You can now use the clock_shift_matrices function with the modification that the MatrixGate objects have qid_shape set to (n,). Here's an example usage:

In [4]:
n = 5
clock_op, shift_op = clock_shift_matrices(n)

print("Clock matrix:\n", cirq.unitary(clock_op))
print("Shift matrix:\n", cirq.unitary(shift_op))


Clock matrix:
 [[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.30901699+0.95105652j  0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.80901699+0.58778525j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.80901699-0.58778525j  0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.30901699-0.95105652j]]
Shift matrix:
 [[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]


Now we would like to construct some multi-qudit gates that entangle two qudits. Gates that usually perform this entangling in quantum computing are the `CZ` or controlled-phase gate, and the `CX` or `CNOT` gates. To generalize these gates we need to construct controlled-shift and controlled-clock operators. Here is the `controlled_clock` function to generalize the `CZ` gate. 

In [5]:
import cirq
import numpy as np

def controlled_clock(n: int, control: cirq.Qid, target: cirq.Qid):
    # Define the clock matrix
    clock_mat = np.zeros((n, n), dtype=np.complex128)
    for i in range(n):
        clock_mat[i, i] = np.exp(2j * np.pi * i / n)
    clock_op = cirq.MatrixGate(clock_mat, qid_shape=(n,))

    # Define the controlled clock gate
    cclock_op = cirq.ControlledGate(clock_op, num_controls=1)

    # Define the circuit
    circuit = cirq.Circuit()
    circuit.append(cclock_op(control, target))

    return circuit



In [6]:
n = 5
control = cirq.LineQubit(0)
target = cirq.LineQid(1, dimension=n)

circuit = controlled_clock(n, control, target)
print(circuit)


0: ─────────@─────────────────────────────────────────────────────────────────────────
            │
            ┌                                                                     ┐
            │ 1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   │
            │ 0.   +0.j     0.309+0.951j  0.   +0.j     0.   +0.j     0.   +0.j   │
1 (d=5): ───│ 0.   +0.j     0.   +0.j    -0.809+0.588j  0.   +0.j     0.   +0.j   │───
            │ 0.   +0.j     0.   +0.j     0.   +0.j    -0.809-0.588j  0.   +0.j   │
            │ 0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.309-0.951j│
            └                                                                     ┘


Here is an alternate implementation. 

In [7]:
import cirq
import numpy as np

def controlled_clock(n: int, control: cirq.Qid, target: cirq.Qid):
    # Define the clock matrix
    clock_mat = np.zeros((n, n), dtype=np.complex128)
    for i in range(n):
        clock_mat[i, i] = np.exp(2j * np.pi * i / n)
    clock_op = cirq.MatrixGate(clock_mat, qid_shape=(n,))

    # Define the controlled clock gate
    cclock_op = cirq.ControlledGate(clock_op, num_controls=1)

    # Define the circuit
    circuit = cirq.Circuit()
    circuit.append(cclock_op(control, target))

    return circuit

# Define the qudit and qubit
n = 5
control = cirq.LineQubit(0)
target = cirq.LineQid(1, dimension=n)

# Create the circuit
circuit = controlled_clock(n, control, target)

# Get the matrix of the circuit
unitary = circuit.unitary()

# Print the matrix
print("Controlled Clock Matrix Gate:")
print(unitary)


Controlled Clock Matrix Gate:
[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          1.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          1.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0

Entangling a qubit and a qutrit ($3$-level system) can be done as follows. 

In [8]:
import cirq
import numpy as np

# Define the qutrit and qubit
qutrit = cirq.LineQid(0, dimension=3)
qubit = cirq.LineQid(1, dimension=2)

# Define the circuit
circuit = cirq.Circuit()

# Apply a Hadamard gate to the qubit to put it in a superposition of |0⟩ and |1⟩
circuit.append(cirq.H(qubit))

# Define a unitary matrix for the controlled-phase gate
cp_matrix = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, np.exp(1j*np.pi/2)]])

# Define a matrix gate using the cp_matrix with qid_shape specified
cp_gate = cirq.MatrixGate(cp_matrix, qid_shape=(2, 3))

# Apply the controlled-phase gate to entangle the qubit and qutrit
circuit.append(cp_gate(qubit, qutrit))

# Print the circuit
print(circuit)


0 (d=3): ───────#2────────────────────────────────────────────
                │
                ┌                                         ┐
                │1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j│
                │0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j│
1 (d=2): ───H───│0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j│───
                │0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j│
                │0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j│
                │0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j│
                └                                         ┘


More generally, we can construct controlled-U gates for random unitary matrices $U \in U(n)$ to entangle qubits with qudits. Given a bipartite dessin d'enfant, we can then construct a perturbation of chimera codes (since controlled-shift and controlled-clock operators are the more standard choice of gates used for entangling). This allows us to study two things. First, it could allow for custom AI-designed entangling gates for specific hardware backends, that might allow for more optimal error correction and computation. This could also be studied as purturbations of surface codes and could be thought of as modelling errors or noise in chimera surface codes. 

In [9]:
import scipy.stats as stats
import cirq

# Define the qubit and qudit
n = 5
control_qubit = cirq.LineQubit(1)
target_qudit = cirq.LineQid(0, dimension=n)

# Generate a random unitary matrix of dimension n
random_unitary = stats.unitary_group.rvs(n)

# Define a gate that applies the random unitary matrix to the target qudit
# random_unitary_gate = cirq.MatrixGate(random_unitary).on(target_qudit)
random_unitary_gate = cirq.MatrixGate(random_unitary, qid_shape=(n,))


# Define a controlled version of the target gate using the control qubit
controlled_gate = cirq.ControlledGate(random_unitary_gate).on(control_qubit, target_qudit)

# Create a circuit and add the controlled target gate to it
circuit = cirq.Circuit()
circuit.append(controlled_gate)

# Print the circuit to verify it
print(circuit)


            ┌                                                                     ┐
            │ 0.269+0.122j -0.099-0.426j -0.723+0.25j   0.337-0.001j  0.144-0.033j│
            │-0.256+0.234j -0.523-0.284j  0.209+0.118j  0.034+0.658j -0.12 +0.14j │
0 (d=5): ───│ 0.15 -0.153j -0.165-0.332j  0.097-0.342j  0.237-0.259j -0.747-0.099j│───
            │-0.136-0.349j  0.457-0.197j  0.238+0.252j  0.49 +0.099j  0.002+0.492j│
            │-0.071-0.776j -0.258+0.051j -0.142-0.3j    0.104+0.261j  0.274-0.245j│
            └                                                                     ┘
            │
1: ─────────@─────────────────────────────────────────────────────────────────────────


From here, we can construct a surface code or graph state using these $2$-qudit gates as entangling operators to build a generalization to surface codes called **"chimera codes"**. 